### Importing modules

In [1]:
from preset_source_models import ModelSourceMaker
from simulate_ALMA_observations import run_simalma, extract_MS_data
from helpers import process_inputs, check_user_FITS_file

### Generating a model source using presets

In [2]:
# Generate and save a flat disk model
factory = ModelSourceMaker(
    fov_arcsec=12.8,
    npix=128,
    freq_Hz=230e9,  # 230 GHz
    chan_width_Hz=50e6,  # 50 MHz
    ra_dec_center="23h59m59.96s -34d59m59.50s"
)
factory.generate_and_save("trial_flat_disk", factory.flat_disk(radius_arcsec=3.84, intensity=2.5))

Source model FITS file saved to: source_model/trial_flat_disk.fits
Image of source model saved as PDF to: source_model/trial_flat_disk.pdf


In [ ]:
# Create a ModelSourceMaker instance
# factory = ModelSourceMaker(
#     fov_arcsec=100,
#     npix=128,
#     ra_dec_center="05h35m17.3s -05d23m28s"
# )

In [ ]:
# Generate and save all model images
########################################################################################################################
# factory.generate_and_save("flat_disk", factory.flat_disk(radius_arcsec=30, intensity=2.0))
# factory.generate_and_save("gaussian", factory.gaussian(fwhm_arcsec=20, amplitude=1.0))
# factory.generate_and_save("elliptical_disk", factory.elliptical_disk(major_arcsec=40, minor_arcsec=20, angle_deg=45))
# factory.generate_and_save("hollow_ring", factory.hollow_ring(inner_radius_arcsec=20, outer_radius_arcsec=40))
# factory.generate_and_save("concentric_rings", factory.concentric_rings(radii_arcsec=[15, 30, 45], widths_arcsec=[2, 5, 7], intensities=[1.0, 0.5, 0.3]))
# factory.generate_and_save("elliptical_gaussian", factory.elliptical_gaussian(major_fwhm_arcsec=30, minor_fwhm_arcsec=20, angle_deg=30, amplitude=1.0))
# factory.generate_and_save("spiral_arms", factory.spiral_arms(arm_width_arcsec=5, pitch_angle_deg=20, number_of_turns=3, intensity=1.0))
# factory.generate_and_save("point_sources", factory.point_sources(source_list=[(0, 0, 1.0), (10, 10, 0.5), (-10, -10, 0.3)]))
########################################################################################################################

In [ ]:
# factory.generate_and_save("elliptical_disk_0", factory.elliptical_disk(major_arcsec=40, minor_arcsec=20, angle_deg=0))
# factory.generate_and_save("elliptical_disk_45", factory.elliptical_disk(major_arcsec=40, minor_arcsec=20, angle_deg=45))
# factory.generate_and_save("elliptical_disk_90", factory.elliptical_disk(major_arcsec=40, minor_arcsec=20, angle_deg=90))
# factory.generate_and_save("elliptical_disk_275", factory.elliptical_disk(major_arcsec=40, minor_arcsec=20, angle_deg=275))
# factory.generate_and_save("elliptical_disk_-25", factory.elliptical_disk(major_arcsec=40, minor_arcsec=20, angle_deg=-25))

### Check the input/generated FITS file

In [3]:
check_user_FITS_file(fits_file="source_model/trial_flat_disk.fits")

True

### Run SIMALMA

In [ ]:
run_simalma(project="trial_flat_disk",
            overwrite=True,
            skymodel="source_model/trial_flat_disk.fits",
            indirection="J2000 23h59m59.96s -34d59m59.50s",
            incell="0.1arcsec",
            inbright="0.004",
            incenter="230.0GHz",
            inwidth="50MHz",
            antennalist=["alma.cycle5.3.cfg"],
            totaltime="1800s",
            mapsize=" ",
            tpnant = 0,
            tptime="0s",
            pwv=0.6,
            dryrun = False,
            image = False
            )

In [ ]:
run_simalma(project="trial_flat_disk",
            overwrite=True,
            skymodel="source_model/trial_flat_disk.fits",
            indirection="",
            incell="",
            inbright="0.004",
            incenter="",
            inwidth="",
            antennalist=["alma.cycle5.3.cfg"],
            totaltime="1800s",
            mapsize=" ",
            tpnant = 0,
            tptime="0s",
            pwv=0.6,
            dryrun = False,
            image = False
            )

### Extract MS data to .npz

In [ ]:
ms_path = "trial_flat_disk/trial_flat_disk.alma.cycle5.3.noisy.ms"
npz_file = "trial_flat_disk.npz"

_ = extract_MS_data(ms_path=ms_path, npz_file=npz_file, make_visibility_plots=True)

# TO DO / INCOMPLETE

### Prepare data for RML imaging

In [26]:
import numpy as np
from astropy.io import fits
from mpol import coordinates, gridding
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

fits_file_path = "source_model/flat_disk.fits"
npz_file = "data_for_rml/flat_disk.npz"

# Read npix and pixel_scale_arcsec from the FITS header
header = fits.getheader(fits_file_path)
npix = header['NAXIS1']  # Assuming square image, NAXIS1 and NAXIS2 are the same
if 'CUNIT1' in header:
    if header['CUNIT1'].lower() == 'arcsec':
        pixel_scale_arcsec = abs(header['CDELT1'])  # Already in arcseconds
    elif header['CUNIT1'].lower() == 'deg':
        pixel_scale_arcsec = abs(header['CDELT1']) * 3600
    elif header['CUNIT1'].lower() == 'rad':
        pixel_scale_arcsec = abs(header['CDELT1']) * 206264.80624709636
    else:
        raise ValueError(f"Unrecognised CUNIT1 value: {header['CUNIT1']}")
else:
    raise ValueError("CUNIT1 not found in the FITS header. Please ensure that the FITS file has the required header keywords.")

print(f"Image properties: npix = {npix}, pixel_scale_arcsec = {pixel_scale_arcsec} arcsec")

# instantiate the gridcoords object
coords = coordinates.GridCoords(cell_size=pixel_scale_arcsec, npix=npix)

# Read the visibility data from the .npz file
npz_data = np.load(npz_file)
uu = npz_data['u']  # u coordinates in meters
vv = npz_data['v']  # v coordinates in meters
weight = npz_data['weight']  # visibility weights
vis_data = npz_data['data']  # visibility data (complex numbers)
data_re = np.real(vis_data) # real part of visibility data
data_im = np.imag(vis_data) # imaginary part of visibility data
# Print the shapes of the extracted data

print(f"Visibility data shapes: uu={uu.shape}, vv={vv.shape}, weight={weight.shape}, data_re={data_re.shape}, data_im={data_im.shape}")

# instantiate the dirty imager object
imager = gridding.DirtyImager(
    coords=coords,
    uu=uu,
    vv=vv,
    weight=weight,
    data_re=data_re,
    data_im=data_im,
)

### Calculating the dirty image and dirty beam and plotting them
###~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~###
# calculate the dirty image and the beam
img, beam, beam_area = imager.get_dirty_image(weighting="uniform", robust=0.0, max_scatter=2.8, get_beam_area=True) # default unit: Jy/beam, can be set to unit='Jy/arcsec^2' also

print(beam.shape)
print(img.shape)
beam_area = beam_area[0]
print('Beam area:', beam_area)

# visualise the calculated dirty image and dirty beam
kw = {"origin": "lower", "interpolation": "none", "extent": imager.coords.img_ext}
fig, ax = plt.subplots(ncols=2, figsize=(6, 3))
ax[0].imshow(beam[0], **kw)
ax[0].set_title("beam")
#ax[0].set_xlim(-0.1, 0.1)
#ax[0].set_ylim(-0.1, 0.1)
img0_for_cbar = ax[1].imshow(img[0], **kw)
ax[1].set_title("image")
#plt.colorbar(img0_for_cbar, ax=ax[1], orientation="vertical", pad=0.01)
# Create a new axis for the colorbar next to ax[1]
divider = make_axes_locatable(ax[1])
cax = divider.append_axes("right", size="5%", pad=0.05)
# Add colorbar to the new axis (cax)
plt.colorbar(img0_for_cbar, cax=cax)

for a in ax:
    a.set_xlabel(r"$\Delta \alpha \cos \delta$ [${}^{\prime\prime}$]")
    a.set_ylabel(r"$\Delta \delta$ [${}^{\prime\prime}$]")
plt.tight_layout()
plt.show()
###~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~###

### Average and export the data to PyTorch dataset
###~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~###
# average the data
averager = gridding.DataAverager(coords=coords, uu=uu, vv=vv, weight=weight, data_re=data_re, data_im=data_im)

# export to PyTorch dataset
dset = averager.to_pytorch_dataset(max_scatter=2.8)
###~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~###


Image properties: npix = 128, pixel_scale_arcsec = 0.0007812500000000041 arcsec
Visibility data shapes: uu=(162540,), vv=(162540,), weight=(162540,), data_re=(162540,), data_im=(162540,)


ValueError: zero-size array to reduction operation minimum which has no identity